In [ ]:
def knn():
    for i in range(1, 2):
        knn = KNeighborsClassifier(n_neighbors=i)

        knn.fit(train_data, train_labels) #train on processed train data

        pred_knn  = knn.predict(dev_data) #predict on dev data

        accuracy_count_knn = 0

        for j in range(pred_knn.shape[0]):
            if pred_knn[j] == dev_labels[j]:
                accuracy_count_knn += 1
            else:
                continue

        print(f'KNN accuracy at i={i} : {(accuracy_count_knn/pred_knn.shape[0])}')

        if i == 1:
            plot_confusion_matrix(dev_labels, pred_knn)
    
#     ########## just type 1 and 2 ##########
    train_data_1_2 = train_data[(train_labels == 1) | (train_labels == 2)]
    train_labels_1_2 = train_labels[(train_labels == 1) | (train_labels == 2)]
    pred_knn_1_2_data = dev_data[(pred_knn == 1) | (pred_knn == 2)]
    pred_knn_1_2_actual_labels = dev_labels[(pred_knn == 1) | (pred_knn == 2)]
    
    
    baseline_models(train_data_1_2, train_labels_1_2, pred_knn_1_2_data, pred_knn_1_2_actual_labels)
    
      
knn()

In [ ]:
def bernoulli_nb(alphas):

    # BernoulliNB training
    model_nb = BernoulliNB()
    model_nb.fit(train_data, train_labels)

    # use cros validation = 5 to validate the alphas
    model_cv = GridSearchCV(estimator=model_nb, param_grid=alphas, cv=5, scoring='accuracy')
    model_cv.fit(train_data, train_labels)

    # create the data frame
    params = [param['alpha'] for param in model_cv.cv_results_["params"]]
    mean_test_score = model_cv.cv_results_["mean_test_score"]
    std_test_score = model_cv.cv_results_["std_test_score"]
    df = pd.DataFrame(list(zip(params, mean_test_score,std_test_score)),
                   columns =['alpha', 'mean_test_score',"std_test_score"])
    print(df)
    
alphas = {'alpha': [1.0e-10, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
bernoulli_nb(alphas)

In [ ]:
def gaussian_nb():


    ####Train on GaussianNB with no var_smoothing####
    model_gnb = GaussianNB()
    model_gnb.fit(train_data,train_labels)

    sigmas = model_gnb.sigma_
    thetas = model_gnb.theta_

    ## predict on dev data

    pred_gnb = model_gnb.predict(dev_data)
    accuracy_count_gnb = 0


    ###generate accuracy
    for j in range(dev_labels.shape[0]):
        if pred_gnb[j] == dev_labels[j]:
            accuracy_count_gnb += 1
        else:
            continue

    print("GaussianNB accuracy w/ no modification: " + str(accuracy_count_gnb/pred_gnb.shape[0]))

    ###Train on gaussianNB with var_smoothing = .1###

    model_gnb_s = GaussianNB(var_smoothing=.1)
    model_gnb_s.fit(train_data,train_labels)

    sigmas2 = model_gnb_s.sigma_
    thetas2 = model_gnb_s.theta_


    ## predict on dev data

    pred_gnb_s = model_gnb_s.predict(dev_data)
    accuracy_count_gnb_s = 0


    ###generate accuracy
    for j in range(dev_labels.shape[0]):
        if pred_gnb_s[j] == dev_labels[j]:
            accuracy_count_gnb_s += 1
        else:
            continue

    print("GaussianNB accuracy w/ var smoothing @ .1: " + str(accuracy_count_gnb_s/pred_gnb_s.shape[0]))

    #train on bernoulliNB

    model_gnb_s2 = BernoulliNB()
    model_gnb_s2.fit(train_data,train_labels)

    pred_gnb_s2 = model_gnb_s2.predict(dev_data)
    accuracy_count_gnb_s2 = 0

    for j in range(dev_labels.shape[0]):
        if pred_gnb_s2[j] == dev_labels[j]:
            accuracy_count_gnb_s2 += 1
        else:
            continue

    print("Bernoulli NB performance: " + str(accuracy_count_gnb_s2/pred_gnb_s2.shape[0]))


    
gaussian_nb()

In [ ]:

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# logistic regression

logistic_result = {}
logistic_result_weights = {}
c_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1]

for c in c_values:
    model_logistic = LogisticRegression(C = c, solver="liblinear", multi_class="auto")
    model_logistic.fit(train_data, train_labels)
    model_logistic_pred = model_logistic.predict(dev_data)
    logistic_result[c] = metrics.accuracy_score(dev_labels, model_logistic_pred)

# find best c
best_c = max(logistic_result, key=logistic_result.get)

# build dataframe of f1_score at different c values
logistic_result_df = pd.DataFrame(data = zip(c_values,list(logistic_result.values())), columns = ['c','accuracy'])
logistic_result_df['regularization'] = 1 / np.array(c_values)

# print results
print('Logistic Regression Classifier:')
print('-'*50)
print(f'Best "c" in logistic regression is: c={best_c}, F1 score={round(logistic_result[best_c],3)}')
print()
print(logistic_result_df[['c','regularization','accuracy']])

In [ ]:
def random_forest(train_data=train_data, 
    train_labels=train_labels, 
    dev_data=dev_data, 
    dev_labels=dev_labels,
    show_confusion_matrix=True):
    # Random Forest
    rf = RandomForestClassifier(random_state=0)
    rf.fit(train_data, train_labels)
    print(f'\nRandom Forest accuracy = {(rf.score(dev_data, dev_labels)*100)}')
    pred_labels  = rf.predict(dev_data)
    if show_confusion_matrix == True:
        plot_confusion_matrix(dev_labels, pred_labels)
        
    print(f'cover type 1 and type 2 total correct {np.sum(np.diag(metrics.confusion_matrix(dev_labels, pred_labels))[:2])}')

random_forest()

In [ ]:
def split_soil_type_features(dataset):
    # get feature indexes for Soil_Types
    other_idx = [i for i, col in enumerate(train.columns) if not 'Soil' in col and 'Cover' not in col]
    soil_type_idx = [i for i, col in enumerate(train.columns) if 'Soil' in col]
    
    soil_type_features = dataset[:, soil_type_idx]
    other_features = dataset[:, other_idx]
    
    return other_features, soil_type_features

In [ ]:
def spca_n_comp_trials(split_features=True):
    print('without spca:')
    baseline_models(
            train_data=train_data, 
            train_labels=train_labels, 
            dev_data=dev_data,
            dev_labels=dev_labels,
            show_confusion_matrix=False
        )
    
    for alpha_set in [1,10,100]:
        for n_comp in range(1,6):
            print()
            print('-'*25)
            print(f'number of components = {n_comp}. alpha = {alpha_set}')

            
            if split_features == True:
                ##### transform train_data #####
                other_features, soil_type_features = split_soil_type_features(train_data)
                spca = SparsePCA(n_components = n_comp, alpha=alpha_set, tol=0.1) # always done now, depreciated #, normalize_components=True)
                spca_soil_features = spca.fit_transform(soil_type_features)
                spca_transformed_train_data = np.concatenate((other_features, spca_soil_features), axis=1)

                ##### transform dev_data #####
                other_features, soil_type_features = split_soil_type_features(dev_data)
                spca = SparsePCA(n_components = n_comp, alpha=alpha_set, tol=0.1) # always done now, depreciated #, normalize_components=True)
                spca_soil_features = spca.fit_transform(soil_type_features)
                spca_transformed_dev_data = np.concatenate((other_features, spca_soil_features), axis=1)
            else:
                ##### transform train_data #####
                spca = SparsePCA(n_components = n_comp, alpha=alpha_set, tol=0.1) # always done now, depreciated #, normalize_components=True)
                spca_transformed_train_data = spca.fit_transform(train_data)

                ##### transform dev_data #####
                spca = SparsePCA(n_components = n_comp, alpha=alpha_set, tol=0.1) # always done now, depreciated #, normalize_components=True)
                spca_transformed_dev_data = spca.fit_transform(dev_data)

            baseline_models(
                train_data=spca_transformed_train_data, 
                train_labels=train_labels, 
                dev_data=spca_transformed_dev_data,
                dev_labels=dev_labels,
                show_confusion_matrix=False
            )
    return

spca_n_comp_trials()

In [ ]:
spca_n_comp_trials(split_features=False)

In [ ]:
def pca_n_comp_trials(split_features=True):
    print('without pca:')
    baseline_models(
            train_data=train_data, 
            train_labels=train_labels, 
            dev_data=dev_data,
            dev_labels=dev_labels,
            show_confusion_matrix=False
        )
    
    for n_comp in range(1,6):
        print()
        print('-'*25)
        print(f'number of components = {n_comp}')

        if split_features == True:
            ##### transform train_data #####
            other_features, soil_type_features = split_soil_type_features(train_data)
            pca = PCA(n_components = n_comp) # always done now, depreciated #, normalize_components=True)
            pca_soil_features = pca.fit_transform(soil_type_features)
            pca_transformed_train_data = np.concatenate((other_features, pca_soil_features), axis=1)

            ##### transform dev_data #####
            other_features, soil_type_features = split_soil_type_features(dev_data)
            pca = PCA(n_components = n_comp) # always done now, depreciated #, normalize_components=True)
            pca_soil_features = pca.fit_transform(soil_type_features)
            pca_transformed_dev_data = np.concatenate((other_features, pca_soil_features), axis=1)
        else:
            ##### transform train_data #####
            pca = PCA(n_components = n_comp) # always done now, depreciated #, normalize_components=True)
            pca_transformed_train_data = pca.fit_transform(train_data)

            ##### transform dev_data #####
            pca = PCA(n_components = n_comp) # always done now, depreciated #, normalize_components=True)
            pca_transformed_dev_data = pca.fit_transform(dev_data)

        baseline_models(
            train_data=pca_transformed_train_data, 
            train_labels=train_labels, 
            dev_data=pca_transformed_dev_data,
            dev_labels=dev_labels,
            show_confusion_matrix=False
        )
    return

pca_n_comp_trials()

In [ ]:
pca_n_comp_trials(split_features=False)

In [ ]:
# get feature indexes for Soil_Types
other_idx = [i for i, col in enumerate(train.columns) if not 'Soil' in col and 'Cover' not in col]
soil_type_idx = [i for i, col in enumerate(train.columns) if 'Soil' in col]

soil_type_features = train_data[:, soil_type_idx]
other_features = train_data[:, other_idx]

In [ ]:
# a handy function for vizualizing principle components as a heatmap
# this allows you to see what dimensions in the 'original space' are
# active

def visualize_pca_comps_heatmap(plot, comps, alpha=None):
    heatmap = plot.pcolor(comps, cmap=plt.cm.Blues)
    

    x_lab = [i for i in range(comps.shape[1])]
    y_lab = [i for i in (range(comps.shape[0]))]
    
    plot.set_xticks(np.arange(comps.shape[1])+0.5, minor=False)
    plot.set_yticks(np.arange(comps.shape[0])+0.5, minor=False)
    
    # want a more natural, table-like display
    plot.invert_yaxis()
    
    plot.set_xticklabels(x_lab, minor=False)
    plot.set_yticklabels(y_lab, minor=False)
    
    if alpha is not None:
        plt.title(f'Heatmap of Sparse PCA (alpha {alpha}) components Rows: components, Cols: Original dimensions. ')
    else:
        plt.title('Heatmap of PCA components Rows: components, Cols: Original dimensions')

In [ ]:
def pca_testing():
    # change alpha and watch as dimensions get dropped
    n_comp = 2
    
    ###############
    plt.figure(figsize=(6, 6))
    alpha_set = 0.1
    spca = SparsePCA(n_components = n_comp, alpha=alpha_set, tol=0.1) # always done now, depreciated #, normalize_components=True)
    spca.fit(soil_type_features)
    p2 = plt.subplot(1, 1, 1)
    visualize_pca_comps_heatmap(p2, spca.components_, alpha_set)
    ###############

    ###############
    plt.figure(figsize=(6, 6))
    alpha_set = 1
    spca = SparsePCA(n_components = n_comp, alpha=alpha_set, tol=0.1) # always done now, depreciated #, normalize_components=True)
    spca.fit(soil_type_features)
    p2 = plt.subplot(1, 1, 1)
    visualize_pca_comps_heatmap(p2, spca.components_, alpha_set)
    ###############

    ###############
    plt.figure(figsize=(6, 6))
    alpha_set = 10
    spca = SparsePCA(n_components = n_comp, alpha=alpha_set, tol=0.1) # always done now, depreciated #, normalize_components=True)
    spca.fit(soil_type_features)
    p2 = plt.subplot(1, 1, 1)
    visualize_pca_comps_heatmap(p2, spca.components_, alpha_set)
    ###############

    ###############
    plt.figure(figsize=(6, 6))
    alpha_set = 100
    spca = SparsePCA(n_components = n_comp, alpha=alpha_set, tol=0.1) # always done now, depreciated #, normalize_components=True)
    spca.fit(soil_type_features)
    p2 = plt.subplot(1, 1, 1)
    visualize_pca_comps_heatmap(p2, spca.components_, alpha_set)
    ###############

pca_testing()